## In this notebook I want to test if the file _esm_utils.py_ works as expected.

In [2]:
import sys, os
import torch
import scipy

git_folder = '/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/Inverse_Folding'
esm_folder = '/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/esm/'
sys.path.insert(1, os.path.join(git_folder, 'model'))
sys.path.insert(1, os.path.join(git_folder, 'util'))

sys.path.insert(1, esm_folder)
import esm
import esm.pretrained as pretrained

from encoded_protein_dataset_new import get_embedding, EncodedProteinDataset_new, EncodedProteinDataset_aux, collate_fn_new#, dynamic_collate_fn

from esm_utils import load_structure, extract_coords_from_structure, get_atom_coords_residuewise
from esm_utils import sample_esm_batch2
from esm_utils import align_esm
from torch.nn.functional import one_hot



#### Let us load the datasets

In [2]:
### IDEA: MSAS PROCEDURE CAN GIVE DIFFERENT OUTPUT SHAPES? ASK
max_msas = 1
#msa_dir = "/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/msas/"
msa_dir = "/home/luchinoprince/split2/"
encoding_dir ="/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings"

train_dataset = EncodedProteinDataset_aux(os.path.join(msa_dir, 'train'), encoding_dir, noise=0.02, max_msas=max_msas)  

/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/Inverse_Folding/util/encoded_protein_dataset_new.py:163: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encodings = torch.tensor(read_encodings(encoding_path, trim=False))


In [3]:
train_dataset.msas_paths

['/home/luchinoprince/split2/train/4x28C01_train.a3m.pt']

In [4]:
train_dataset[0][0].shape

/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/Inverse_Folding/util/encoded_protein_dataset_new.py:187: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encodings = torch.tensor(read_encodings(encoding_path, trim=False))


torch.Size([9894, 119])

In [5]:
pdb_id = train_dataset.msas_paths[0].split('/')[-1][0:7]
pdb_dir = '/home/luchinoprince/dompdb'
pdb_path = os.path.join(pdb_dir, pdb_id)
structure =  load_structure(pdb_path)
coords, native_seq = extract_coords_from_structure(structure)

/home/luchinoprince/anaconda3/envs/IF/lib/python3.9/site-packages/biotite/structure/io/pdb/file.py:484: UserWarning: 882 elements were guessed from atom_name.
  warnings.warn(


In [6]:
device='cpu'
model, alphabet_esm = pretrained.esm_if1_gvp4_t16_142M_UR50()
model.eval();
model.to(device)
rep = esm.inverse_folding.util.get_encoder_output(model, alphabet_esm, coords)

/home/luchinoprince/anaconda3/envs/IF/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/luchinoprince/anaconda3/envs/IF/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/esm/esm/pretrained.py:174: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


In [7]:
rep

tensor([[ 0.2971,  0.2332,  0.0322,  ...,  0.2790,  0.2932, -0.1578],
        [-0.9711, -0.3768, -0.6929,  ...,  0.5163,  0.2923,  0.3246],
        [-0.9947, -0.5252, -1.0081,  ...,  0.2758,  0.1979,  0.1778],
        ...,
        [-0.5383,  0.6801, -0.5247,  ...,  0.5967,  0.6209,  0.3422],
        [ 0.9668,  0.3003, -0.4380,  ..., -0.5450, -0.0064, -0.3078],
        [ 0.6466,  0.0069,  0.2429,  ..., -0.2576,  0.2538,  0.0252]],
       grad_fn=<SelectBackward0>)

In [8]:
coords.shape

(119, 3, 3)

In [9]:
samples_esm_str = sample_esm_batch2(model, coords, device=0)

#### Now let us test the re-alignment.

In [10]:
msa_train = train_dataset[0][0]
msa_train.shape

/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/Inverse_Folding/util/encoded_protein_dataset_new.py:187: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encodings = torch.tensor(read_encodings(encoding_path, trim=False))


torch.Size([9894, 119])

In [12]:
samples_esm_aligned = align_esm(samples_esm_str, msa_train)

119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119


## Let us try to test the iterator to get the aggregate correlations

In [4]:
def compute_covariance(msa, q):
    """
    Compute covariance matrix of a given MSA having q different amino acids
    """
    M, N = msa.shape

    # One hot encode classes and reshape to create data matrix
    D = torch.flatten(one_hot(msa, num_classes=q), start_dim=1).to(torch.float32)

    # Remove one amino acid
    D = D.view(M, N, q)[:, :, :q-1].flatten(1)

    # Compute bivariate frequencies
    bivariate_freqs = D.T @ D / M
    
    # Compute product of univariate frequencies
    univariate_freqs = torch.diagonal(bivariate_freqs).view(N*(q-1), 1) @ torch.diagonal(bivariate_freqs).view(1, N*(q-1))

    return bivariate_freqs - univariate_freqs

q=21
embeddings = get_embedding(q)

In [5]:
### IDEA: MSAS PROCEDURE CAN GIVE DIFFERENT OUTPUT SHAPES? ASK
max_msas = None
#msa_dir = "/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/msas/"
msa_dir = "/home/luchinoprince/split2/test"
encoding_dir ="/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings"

train_dataset = EncodedProteinDataset_aux(os.path.join(msa_dir, 'structure'), encoding_dir, noise=0.0, max_msas=max_msas)  

/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/4xhyA00.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/test/structure/4xhyA00.a3m.pt


In [6]:
import warnings
warnings.filterwarnings("ignore")
device= 0
model, alphabet_esm = pretrained.esm_if1_gvp4_t16_142M_UR50()
model.eval();
model.to(device)

Ns = torch.zeros(train_dataset.__len__())
Ms = torch.zeros(train_dataset.__len__())
Ms_sampled = torch.zeros(train_dataset.__len__())
corr_esm = torch.zeros(train_dataset.__len__())
resexp = {}

#with torch.no_grad():
#    rep = esm.inverse_folding.util.get_encoder_output(model, alphabet_esm, coords)
it = 0
for filename in train_dataset.msas_paths[0:1]:

    ### LOAD COORDINATES
    pdb_id = train_dataset.msas_paths[it].split('/')[-1][0:7]
    pdb_dir = '/home/luchinoprince/dompdb'
    pdb_path = os.path.join(pdb_dir, pdb_id)
    structure =  load_structure(pdb_path)
    coords, native_seq = extract_coords_from_structure(structure)

    ### Get the samples
    samples_esm_str = sample_esm_batch2(model, coords, device=0)

    ##Now

    ### ALIGN THE SAMPLES
    ## Here move model to CPU
    msa = torch.tensor(train_dataset[it][0], dtype=torch.long)
    M,N=msa.shape
    samples_esm_aligned = torch.tensor(align_esm(samples_esm_str, msa), dtype=torch.long)
    if len(samples_esm_aligned>0):
        ### Compute covariances and correlations
        cov_esm = compute_covariance(samples_esm_aligned,q)
        cov_true = compute_covariance(msa, q)
        res = scipy.stats.pearsonr(cov_true.flatten(), cov_esm.flatten())
        Ms_sampled[it]=len(samples_esm_aligned)
    else:
        res = [0] ### Later check the zeros
        Ms_sampled[it] = 0

    ## Save results
    Ns[it] = N
    Ms[it] = M
    corr_esm[it] = res[0]

    resexp['Ms'] = Ms
    resexp['Ns'] = Ns
    resexp['Ms_sampled'] = Ms_sampled
    resexp['corr_esm'] = corr_esm
    torch.save(resexp, "res_corr_esm_structure.pkl")
    with open("tracker.txt", "w") as f:
        line = "fineshed " + str(it) + "\n"
        f.write(line)
    it+=1



    



104m position 104 out of 104, batch 1 out of 1
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
1

In [11]:
from collections import defaultdict

alphabet='ACDEFGHIKLMNPQRSTVWY-'
default_index = alphabet.index('-')
aa_index = defaultdict(lambda: default_index, {alphabet[i]: i for i in range(len(alphabet))})
aa_index_inv = dict(map(reversed, aa_index.items()))

In [4]:
#check=torch.load("res_corr_esm_structure.pkl")

In [ ]:
samples_esm_aligned

In [19]:
samples_esm_aligned.shape

torch.Size([900, 104])